#### Example of Bayesian updating

We can show a simple example. Imagine trying to estimate the altitude of a plane attempting to land near an airstrip. We receive radar altitude measurements; these are inaccurate and occasionally. We have a rough idea of how high the plane might be (it's not likely to be trying to land from 10 km up), and very rough idea of how fast it is descending.

If we can define ways of sampling from these distributions, we can then update our beliefs. We will do this by "copying" samples which correspond to more likely states. This turns out to be a well-founded algorithm for Bayesian estimation.

In [ ]:

def redraw_figure(fig):
    IPython.display.clear_output(wait=True)
    IPython.display.display(fig)
    time.sleep(0.25)
    
    
def estimate_landing(iters=100, noise=0.1, k=5):
    n_samples = 50
    alt_guess = np.random.normal(500, 500, n_samples)
    vsi_guess = np.random.normal(-50, 10, n_samples)
    
    
    alt_true = 500
    vsi_true = -47.5
    
    # set up a figure
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.set_ylim(-0.5, 700)
    ax.set_ylabel("Altitude (m)")
    ax.set_xlabel("Time (seconds)")
    ax.set_xlim(-1, iters+1)
    
    for ix in range(iters):
    
        
        # make a noisy observation
        alt_obs = alt_true + np.random.normal(0, noise)
        
        # update our "true" model
        alt_true += vsi_true
        vsi_true *= 0.95
        
        ## LIKELIHOOD
        # compute a weighting function 
        u_weights = np.exp(-(alt_guess - alt_obs)**2 * k)
        
        
        weights = u_weights / np.sum(u_weights) # normalise
        
        
        ax.text(ix, alt_true, '✈', size=50, horizontalalignment='center', color='C3', alpha=0.2)
        ax.set_title("True")
        redraw_figure(fig)
        #ax.plot([ix, ix], [alt_obs-2*noise, alt_obs+2*noise], 'C2')
        #ax.plot([ix, ix], [alt_obs-noise, alt_obs+noise], 'C2', lw=4)
        ax.plot(ix, alt_obs, 'C4o', markersize=20, alpha=0.8)
        ax.set_title("Observed")
        redraw_figure(fig)
        ax.scatter(x=np.tile(ix, (len(alt_guess))), y=alt_guess, s=u_weights*100, 
                   color='C0', zorder=10, edgecolor='black', alpha=0.5)
        ax.set_title("Estimates")
        redraw_figure(fig)
        
        ax.plot(ix, np.sum(alt_guess * weights), 'C1o', markersize=20, zorder=100)
        redraw_figure(fig)
        
        
        ## SAMPLING        
        # copy the particles, favouring heavily weighted ones
        copies = pfilter.resample(weights)
        
        alt_guess = alt_guess[copies]
        vsi_guess = vsi_guess[copies]
        
        # APPLY FUNCTION TO SAMPLES
        # update our position estimate
        # internal model
        alt_guess += vsi_guess
        vsi_guess *= 0.95
        
        # DIFFUSE              
        # add a little noise
        alt_guess += np.random.normal(0, 20, alt_guess.shape)
        vsi_guess += np.random.normal(0, 2, vsi_guess.shape)
        #ax.scatter(x=np.tile(ix+0.5, (len(alt_guess))), y=alt_guess, 
        #           s=10, color='C2', zorder=10)
        #ax.set_title("Predict")
        #redraw_figure(fig)
        
        
        
        ax.set_ylim(-0.5, 700)
        ax.set_ylabel("Altitude (m)")
        ax.set_xlabel("Time (seconds)")
        ax.set_xlim(-1, iters+1)
                
        
estimate_landing(noise=100, k=8e-5, iters=10)  

In [ ]:

def estimate_coin(observations):
    n_samples = 20
    bias_samples = np.random.uniform(0, 1, n_samples)
    k = 5
    
    # set up a figure
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.set_ylim(-0.5, 1.5)
    ax.set_xlim(-1, len(observations)+1)
    
    for ix, observed in enumerate(observations):        
        ax.plot(ix, observed, 'C1d')
        
        
        # guess what we *would* observe
        guessed_observations = np.random.binomial(n=1, p=np.tile(bias_samples, (50, 1)))
        print(guessed_observations.shape)
        
        # compute a weighting function
        weights = np.sum(np.exp(-(observed - guessed_observations)**2 * k), axis=0)
        weights = weights / np.sum(weights) # normalise
        
        ax.scatter(x=np.tile(ix, (len(bias_samples))), y=bias_samples, s=weights*500, color='C0')
        # copy the particles, favouring heavily weighted ones
        copies = pfilter.resample(weights)
        
        for i in range(len(bias_samples)):
            ax.plot([ix, ix+1], [bias_samples[i], bias_samples[copies[i]]], 'C0-', alpha=0.2)
        
        bias_samples = bias_samples[copies]
                        
        
        # add a little noise
        bias_samples = np.clip(bias_samples + np.random.normal(0,0.01,n_samples), 0, 1.0)
                
        
estimate_coin(np.random.binomial(n=1, p=0.75, size=30))        
    